## Mission To Mars Notebook


### Dependencies

In [98]:
#--Dependencies

from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import numpy as np
import time
from urllib.parse import urlsplit
import pymongo

### Web Scrape Code

In [99]:
#Chromedriver and Browser set-up

driver_path = {"executable_path":"chromedriver.exe"}
browser =  Browser('chrome', **driver_path, headless=False)

#### NASA Mars News

In [100]:
#URL set-up

url = "https:/mars.nasa.gov/news/"
browser.visit(url)

In [101]:
# Beautiful Soup Html parser set-up

html = browser.html
soup = bs(html, "html.parser")

In [102]:
# Fetch latest news title and paragraph text

news_title = soup.find("div", class_= "content_title").text
news_p = soup.find("div", class_= "article_teaser_body").text


print(f"Latest Title: {news_title}")
print("----------------------------")
print(f"Title Paragraph: {news_p}")

Latest Title: For InSight, Dust Cleanings Will Yield New Science
----------------------------
Title Paragraph: Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


#### JPL Mars Space Images - Featured Image

In [103]:
image_url = "https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA09320-1920x1200.jpg"
browser.visit(image_url)


#### Mars Weather

In [104]:
#Get the latest tweet from mar's weather twitter account

tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)

In [105]:
#Turn into soup object for text
html_tweet = browser.html
soup = bs(html_tweet, "html.parser")

mars_weather = soup.find("p", class_= "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

mars_weather = mars_weather.replace("\n", " ")
mars_weather = mars_weather[0:-26]
mars_weather

'InSight sol 156 (2019-05-05) low -99.2ºC (-146.6ºF) high -18.1ºC (-0.6ºF) winds from the SW at 4.7 m/s (10.5 mph) gusting to 13.8 m/s (30.8 mph) pressure at 7.40 hPa'

#### Mars Facts

In [106]:
facts_url = "https://space-facts.com/mars/"

facts_table = pd.read_html(facts_url)
facts_table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [75]:
#--Turn into DataFrame

#facts_df = pd.DataFrame(facts_table[0])

#facts_df.columns = ["Measurements", "Values"]
#facts_df.reset_index(["Measurements"], inplace=True)

facts_df


,Values
Measurements,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [76]:
#turn into HTML table

facts_table = facts_df.to_html()
facts_table = facts_table.replace("\n", "")
facts_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Measurements</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

#### Mars Hemispheres (Images)

In [77]:
#set-url and launch browser

hem_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hem_url)

In [90]:
#get base url

hem_base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(hem_url))
print(hem_base_url)


#Soup object
hem_html = browser.html

soup = bs(hem_html, "html.parser")

#Find all divs containing class 'item'

div_data = soup.find_all("div", class_="item")



https://astrogeology.usgs.gov/


In [79]:
# create image list to append url to

hem_img_urls = []
hem_img_urls


[]

In [96]:
#--loop and find images
for item in div_data:

    #Find and store title
    img_title = item.find("h3").text
    #print(title)
    
    #Find and store link to image 
    img_url = item.find("a", class_= "itemLink product-item")["href"]
    
    #Visit image web page
    browser.visit(hem_base_url + img_url)
    
    #Convert image web page to html and then soup object
    img_html = browser.html

    soup = bs(img_html, "html.parser")
    
    #Get full image source
    img_src = hem_base_url + soup.find("img", class_="wide-image")["src"]
    
    #Append to hem_img_urls list
    hem_img_urls.append({"Image Title": img_title, "Image URL": img_src})
    


In [97]:
#--Pring hem_img_urls

hem_img_urls

[{'Image Title': 'Cerberus Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'Image Title': 'Schiaparelli Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'Image Title': 'Syrtis Major Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'Image Title': 'Valles Marineris Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]